# XLM-RoBERTa Fine-tuning for Kyrgyz Punctuation Restoration

Fine-tunes `xlm-roberta-base` on the Kyrgyz punctuation dataset.

**Same split & setup as mBERT** for fair comparison.

**Before running:** Runtime → Change runtime type → **T4 GPU**

In [ ]:
!pip install -q transformers datasets seqeval accelerate scikit-learn

In [ ]:
from google.colab import files
import json
import os

if not os.path.exists('train_data.json'):
    uploaded = files.upload()
    print("File uploaded!")
else:
    print("File already exists.")

with open('train_data.json', 'r') as f:
    raw_data = json.load(f)

print(f"Total sentences: {len(raw_data)}")

In [ ]:
import re
from collections import Counter

LABEL2ID = {'O': 0, 'COMMA': 1, 'PERIOD': 2, 'QUESTION': 3}
ID2LABEL = {v: k for k, v in LABEL2ID.items()}
NUM_LABELS = len(LABEL2ID)

def extract_tokens_and_labels(text):
    raw_tokens = text.split()
    words = []
    labels = []
    for token in raw_tokens:
        word = token.rstrip('.,?!;:\'\"\u201c\u201d\u2013\u2014\u2026()[]{}\u00ab\u00bb')
        if not word:
            continue
        trailing = token[len(word):]
        label = 'O'
        for char in trailing:
            if char == '.':
                label = 'PERIOD'
                break
            elif char == ',':
                label = 'COMMA'
                break
            elif char == '?':
                label = 'QUESTION'
                break
        words.append(word)
        labels.append(label)
    return words, labels

all_words = []
all_labels = []
skipped = 0

for entry in raw_data:
    text = entry['text'][0]
    words, labels = extract_tokens_and_labels(text)
    if len(words) < 2:
        skipped += 1
        continue
    all_words.append(words)
    all_labels.append(labels)

print(f"Processed: {len(all_words)} sentences, skipped: {skipped}")
flat_labels = [l for labels in all_labels for l in labels]
print(f"Label distribution: {Counter(flat_labels)}")

In [ ]:
from sklearn.model_selection import train_test_split

# SAME split as mBERT for fair comparison
train_words, test_words, train_labels, test_labels = train_test_split(
    all_words, all_labels, test_size=0.15, random_state=42
)

train_words, val_words, train_labels, val_labels = train_test_split(
    train_words, train_labels, test_size=0.1, random_state=42
)

print(f"Train: {len(train_words)}, Val: {len(val_words)}, Test: {len(test_words)}")

In [ ]:
from transformers import AutoTokenizer
import torch
from torch.utils.data import Dataset

MODEL_NAME = "xlm-roberta-base"
MAX_LEN = 256

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

class PunctDataset(Dataset):
    def __init__(self, words_list, labels_list, tokenizer, max_len):
        self.words_list = words_list
        self.labels_list = labels_list
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.words_list)

    def __getitem__(self, idx):
        words = self.words_list[idx]
        labels = self.labels_list[idx]

        encoding = self.tokenizer(
            words,
            is_split_into_words=True,
            truncation=True,
            max_length=self.max_len,
            padding='max_length',
            return_tensors='pt'
        )

        word_ids = encoding.word_ids(batch_index=0)

        # Assign label to LAST subtoken of each word
        label_ids = [-100] * len(word_ids)
        for i in range(len(word_ids) - 1, -1, -1):
            wid = word_ids[i]
            if wid is None:
                continue
            if i == len(word_ids) - 1 or word_ids[i + 1] != wid:
                label_ids[i] = LABEL2ID[labels[wid]]

        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(label_ids, dtype=torch.long)
        }

train_dataset = PunctDataset(train_words, train_labels, tokenizer, MAX_LEN)
val_dataset = PunctDataset(val_words, val_labels, tokenizer, MAX_LEN)
test_dataset = PunctDataset(test_words, test_labels, tokenizer, MAX_LEN)

print(f"Datasets: Train={len(train_dataset)}, Val={len(val_dataset)}, Test={len(test_dataset)}")

sample = train_dataset[0]
print(f"Input shape: {sample['input_ids'].shape}")
print(f"Non-ignored labels: {(sample['labels'] != -100).sum().item()}")

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from sklearn.metrics import classification_report
import numpy as np

model = AutoModelForTokenClassification.from_pretrained(
    MODEL_NAME,
    num_labels=NUM_LABELS,
    id2label=ID2LABEL,
    label2id=LABEL2ID,
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)

    true_labels = []
    pred_labels = []
    for i in range(labels.shape[0]):
        for j in range(labels.shape[1]):
            if labels[i][j] != -100:
                true_labels.append(ID2LABEL[labels[i][j]])
                pred_labels.append(ID2LABEL[preds[i][j]])

    report = classification_report(
        true_labels, pred_labels,
        labels=['O', 'COMMA', 'PERIOD', 'QUESTION'],
        output_dict=True
    )

    return {
        'f1_weighted': report['weighted avg']['f1-score'],
        'precision_weighted': report['weighted avg']['precision'],
        'recall_weighted': report['weighted avg']['recall'],
    }

training_args = TrainingArguments(
    output_dir='./xlmr_punct_kyrgyz',
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=5e-5,
    weight_decay=0.01,
    warmup_ratio=0.1,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='f1_weighted',
    greater_is_better=True,
    fp16=True,
    logging_steps=50,
    report_to='none',
    seed=42,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

print("Starting XLM-RoBERTa training...")
trainer.train()

In [ ]:
from torch.utils.data import DataLoader

model.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

all_true = []
all_pred = []

test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels']

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        preds = torch.argmax(outputs.logits, dim=-1).cpu()

        for i in range(labels.shape[0]):
            for j in range(labels.shape[1]):
                if labels[i][j] != -100:
                    all_true.append(ID2LABEL[labels[i][j].item()])
                    all_pred.append(ID2LABEL[preds[i][j].item()])

print(f"Total test tokens: {len(all_true)}")
print()
print("=" * 60)
print("XLM-RoBERTa CLASSIFICATION REPORT")
print("=" * 60)
print(classification_report(
    all_true, all_pred,
    labels=['O', 'COMMA', 'PERIOD', 'QUESTION'],
    digits=3
))

In [ ]:
report = classification_report(
    all_true, all_pred,
    labels=['O', 'COMMA', 'PERIOD', 'QUESTION'],
    digits=3,
    output_dict=True
)

w = report['weighted avg']
print("\n" + "=" * 60)
print("COPY THESE NUMBERS FOR THE PAPER:")
print("=" * 60)
print(f"XLM-RoBERTa (fine-tuned)  &  {w['precision']:.3f}  &  {w['recall']:.3f}  &  {w['f1-score']:.3f}")
print()
print("Per-class results:")
for cls in ['O', 'COMMA', 'PERIOD', 'QUESTION']:
    c = report[cls]
    print(f"  {cls}: P={c['precision']:.3f}  R={c['recall']:.3f}  F1={c['f1-score']:.3f}  Support={c['support']}")
print()
print("LaTeX table row:")
print(f"XLM-RoBERTa (fine-tuned) & {w['precision']:.3f} & {w['recall']:.3f} & {w['f1-score']:.3f} \\\\")

## Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

labels = ['O', 'COMMA', 'PERIOD', 'QUESTION']

# Normalized confusion matrix
cm_norm = confusion_matrix(all_true, all_pred, labels=labels, normalize='true')

fig, ax = plt.subplots(figsize=(6, 5))
disp = ConfusionMatrixDisplay(confusion_matrix=cm_norm, display_labels=labels)
disp.plot(ax=ax, cmap='Oranges', values_format='.3f')
ax.set_title('XLM-RoBERTa — Confusion Matrix (Normalized)', fontsize=13)
plt.tight_layout()
plt.savefig('confusion_matrix_xlmr.png', dpi=300, bbox_inches='tight')
plt.show()
print("Saved: confusion_matrix_xlmr.png")

# Raw counts
cm_raw = confusion_matrix(all_true, all_pred, labels=labels)
print("\nRaw counts:")
print(cm_raw)

In [ ]:
files.download('confusion_matrix_xlmr.png')